In [1]:
import numpy as np
import cv2
import glob
import math

In [154]:
# variable to store which video we must elaborate, we must insert only numbers 1,2,3 or 4
videoToElaborate = 4

# get list of frames of video 1
listImages = glob.glob("framesVideo" + str(videoToElaborate) + "/uncalibrated/*.jpg")

# for each image
for image in listImages:
    # open the actual image in color mode
    img1 = cv2.imread(image, cv2.IMREAD_COLOR)
    imgtemp = cv2.imread(image, 0)
    img13 = cv2.imread(image, cv2.IMREAD_COLOR)
    imgw = cv2.imread(image, cv2.IMREAD_COLOR)

    # dimension of the object plus a small piece of the glass
    img = img1[:, int(1920 / 6):int(1920 / 2) + int(1920 / 30), :]

    img5 = img[:, 509:736, :]

    # white background and black object
    mask2 = cv2.inRange(img, (40, 0, 0), (173, 120, 96))

    # blue background and black object
    target = cv2.bitwise_and(img, img, mask=mask2)

    #target = cv2.erode(target, kernel, iterations=5)

    # white object and black background
    #target = cv2.inRange ( target , (0,0,0) , (0,0,0) )

    # If pixel intensity is greater than the set threshold, value set to 0 black, else set to 255 white
    #_ , target = cv2.threshold ( target , 0 , 255 , cv2.THRESH_BINARY_INV )

    img13[:, int(1920 / 6):int(1920 / 2) + int(1920 / 30)] = target

    #yellow glass filter

    #_ , threshold_gray_actual_image = cv2.threshold ( img_w , 150 , 180 , cv2.THRESH_BINARY )

    #hsv_img = cv2.cvtColor(img_w, cv2.COLOR_BGR2YUV)
    mask3 = cv2.inRange(img5, (90, 170, 170), (170, 255, 255))

    # this case works
    if videoToElaborate == 1:

        # i know that using erosion i lost a very small bottom part of the object and it is due to the erode and dilate
        # methods that i used to delete the label from the object loosing quality but so i obtain a good 3d model.

        mask3 = cv2.inRange(img5, (0, 165, 160), (110, 255, 255))
        mask4 = cv2.inRange(img5, (160, 165, 160), (190, 255, 255))

        mask3 = cv2.bitwise_or(mask3, mask4)
        target1 = cv2.bitwise_and(img5, img5, mask=mask3)
        kernel = np.ones((3, 3), np.uint8)
        target1 = cv2.erode(target1, kernel, iterations=10)
        target1 = cv2.dilate(target1, kernel, iterations=25)
        target1 = cv2.erode(target1, kernel, iterations=3)

        target = cv2.erode(target, kernel, iterations=10)

    elif videoToElaborate == 2:

        # I know very well that using erosion and dilation the image loses quality.
        # Otherwise if I didn't I couldn't highlight the legs of the animal
        # in black at 100% and there were empty holes in the middle.
        # Then another important thing is the black border of the glass, i can delete it using dilate method
        # but i loss precision in the legs. And if i delete that color from the range, some pixel of the legs
        # of the animal are lost. And this border is deleted from the 3d model thanks to the algorithm

        mask3 = cv2.inRange(img5, (0, 185, 160), (110, 255, 255))
        kernel = np.ones((3, 3), np.uint8)
        #mask4 = cv2.inRange(img5, (160, 165, 160), (190, 255, 255))
        kernel = np.ones((3, 3), np.uint8)
        target1 = cv2.bitwise_and(img5, img5, mask=mask3)
        target1 = cv2.dilate(target1, kernel, iterations=8)
        target1 = cv2.erode(target1, kernel, iterations=12)
        target1 = cv2.dilate(target1, kernel, iterations=6)


    elif videoToElaborate == 3:
        mask3 = cv2.inRange(img5, (0, 140, 150), (110, 255, 255))
        target1 = cv2.bitwise_and(img5, img5, mask=mask3)

        # i know that using erosion i lost a very small bottom part of the object and it is due to the erode and dilate
        # methods that i used to delete the label from the object loosing quality but so i obtain a good 3d model.
        # one defect of this algorithm is that the label in the back part of the object is of the same color of the background
        # and to delete it, we lose quality and there is a small hole in the 3d model but for the target of this project that's okay.

        kernel = np.ones((3, 3), np.uint8)
        target1 = cv2.erode(target1, kernel, iterations=3)
        target1 = cv2.dilate(target1, kernel, iterations=8)
        target1 = cv2.erode(target1, kernel, iterations=3)
        target1 = cv2.dilate(target1, kernel, iterations=5)
        target1 = cv2.erode(target1, kernel, iterations=5)

        target = cv2.erode(target, kernel, iterations=10)
        target = cv2.dilate(target, kernel, iterations=8)


    elif videoToElaborate == 4:
        #mask3 = cv2.inRange(img5, (0, 180, 170), (180, 255, 255))
        mask3 = cv2.inRange(img5, (0, 155, 155), (110, 255, 255))
        target1 = cv2.bitwise_and(img5, img5, mask=mask3)
        kernel = np.ones((3,3), np.uint8)

        target1 = cv2.erode(target1, kernel, iterations=3)
        target1 = cv2.dilate(target1, kernel, iterations=8)
        target1 = cv2.erode(target1, kernel, iterations=3)
        target1 = cv2.dilate(target1, kernel, iterations=8)
        target1 = cv2.erode(target1, kernel, iterations=7)

        target = cv2.erode(target, kernel, iterations=4)
        target = cv2.dilate(target, kernel, iterations=3)

    #kernel = np.ones((3, 3), np.uint8)
    #target1 = cv2.dilate(target1, kernel, iterations=5)
    #target1 = cv2.erode(target1, kernel, iterations=3)
    #img_w = img13 [ int ( 1080 / 8 * 1 ) :int ( 1080 / 8 * 7 ) , int ( 1920 / 2-(1920/16)) :int ( 1920 / 2 ), : ]

    #mask3 = cv2.inRange ( img_w , (0,150,160) , (130,255,255) )

    #kernel = np.ones((3,3), np.uint8)
    #target1 = cv2.dilate(target1, kernel, iterations=3)
    yellow_filter = image.replace("uncalibrated", "object")
    cv2.imwrite(yellow_filter, target)

    yellow_filter = image.replace("uncalibrated", "glass")
    cv2.imwrite(yellow_filter, target1)

    #target = cv2.bitwise_xor ( target, target1 , mask = None )

    # #kernel = np.ones ( (10,10) , np.uint8 )
    #target = cv2.erode ( target , kernel )

    for y in range(0, 1080):
        for x in range(int(1920 / 6), int(1920 / 2) + int(1920 / 30)):

            if int(1920 / 6) + 509 <= x <= int(1920 / 2) + int(1920 / 30):

                if target[y][x - 320][0] == 0 and target[y][x - 320][1] == 0 and target[y][x - 320][2] == 0:

                    if target1[y][x - (int(1920 / 6) + 509)][0] == 0 and target1[y][x - (int(1920 / 6) + 509)][1] == 0 and target1[y][x - (int(1920 / 6) + 509)][2] == 0:
                        imgw[y, x] = (0, 0, 0)

            elif int(1920 / 6) <= x <= int(1920 / 6) + 509:

                if target[y][x - 320][0] == 0 and target[y][x - 320][1] == 0 and target[y][x - 320][2] == 0:
                    imgw[y, x] = (0, 0, 0)

    yellow_filter = image.replace("uncalibrated", "backgroundremoved")
    cv2.imwrite(yellow_filter, imgw)


810
